In [1]:
import pandas as pd
import numpy as np
data = pd.read_excel("./work.xlsx")
data 

,属性(指标),唐山市,秦皇岛市,沧州市
0,GDP(亿元),6890.0,1612.0,3588.0
1,GDP指数(上年=100),107.3,106.7,106.9


In [2]:
data=data.T
data=data.reset_index()
data=pd.DataFrame(data=data.values[1:],columns=data.values[0])
data.set_index(['属性(指标)'], inplace=True)
data

,GDP(亿元),GDP指数(上年=100)
属性(指标),,
唐山市,6890,107.3
秦皇岛市,1612,106.7
沧州市,3588,106.9


### 统一量纲标准化

In [3]:
m, n = data.shape  # 获取行数m和列数n
# 熵权法计算
def My_Stander(data1):  # 矩阵标准化(min-max标准化)
    for i in data1.columns:
        for j in range(n+1):
            if i == str(f'X{j}负'):  # 负向指标
                data1[i] = (np.max(data1[i])-data1[i]) / \
                    (np.max(data1[i])-np.min(data1[i]))
            else:  # 正向指标
                data1[i] = (data1[i]-np.min(data1[i])) / \
                    (np.max(data1[i])-np.min(data1[i]))
    return data1
stander_data= My_Stander(data)  # 标准化矩阵

In [4]:
formater="{0:.04f}".format
stander_data=stander_data.applymap(formater)
stander_data

,GDP(亿元),GDP指数(上年=100)
属性(指标),,
唐山市,1.0000,1.0000
秦皇岛市,0.0000,0.0000
沧州市,0.3744,0.3333


In [5]:
stander_data

,GDP(亿元),GDP指数(上年=100)
属性(指标),,
唐山市,1.0000,1.0000
秦皇岛市,0.0000,0.0000
沧州市,0.3744,0.3333


In [6]:
None_ij = [[None] * n for i in range(m)]  # 新建空矩阵
def entropy(data):  # 计算熵值
    data = np.array(data,dtype='float')
    E = np.array(None_ij)
    for i in range(m):
        for j in range(n):
            if data[i][j] == 0:
                e_ij = 0.0
            else:
                P_ij = data[i][j] / data.sum(axis=0)[j]  # 计算比重
                e_ij = (-1 / np.log(m)) * P_ij * np.log(P_ij)
            E[i][j] = e_ij
    E_j = E.sum(axis=0)
    G_j = 1 - E_j
    W_j = G_j / sum(G_j)
    return np.around(np.array(W_j,dtype="float32"),1)
W = entropy(stander_data)  

In [7]:
np.array([0.48889803892358075 ,0.5111019610764193])

array([0.48889804, 0.51110196])

In [8]:
np.around(np.array([0.48889803892358075 ,0.5111019610764193]),1)

array([0.5, 0.5])

In [9]:
stander_data=stander_data.T
save_stander_data=stander_data.copy()
save_stander_data['权重']=W
save_stander_data.to_excel('./标准化后的数据.xlsx')
save_stander_data

属性(指标),唐山市,秦皇岛市,沧州市,权重
GDP(亿元),1.0000,0.0000,0.3744,0.5
GDP指数(上年=100),1.0000,0.0000,0.3333,0.5


In [10]:
stander_data

属性(指标),唐山市,秦皇岛市,沧州市
GDP(亿元),1.0000,0.0000,0.3744
GDP指数(上年=100),1.0000,0.0000,0.3333


In [11]:
W_j = W
# TOPSIS计算
stan_data = np.array(stander_data.T,dtype='float')  # stan_data为标准化矩阵
Z_ij = np.array(None_ij)  # 空矩阵
for i in range(m):
    for j in range(n):
        Z_ij[i][j] = stan_data[i][j]*W_j[j]  # 计算加权标准化矩阵Z_ij
Imax_j = Z_ij.max(axis=0)  # 最优解
Imin_j = Z_ij.min(axis=0)  # 最劣解
Dmax_ij = np.array(None_ij)  # 空矩阵
Dmin_ij = np.array(None_ij)  # 空矩阵
for i in range(m):
    for j in range(n):
        Dmax_ij[i][j] = (Imax_j[j] - Z_ij[i][j]) ** 2
        Dmin_ij[i][j] = (Imin_j[j] - Z_ij[i][j]) ** 2
Dmax_i = Dmax_ij.sum(axis=1)**0.5  # 最优解欧氏距离
Dmin_i = Dmin_ij.sum(axis=1)**0.5  # 最劣解欧氏距离
score = Dmin_i/(Dmax_i+Dmin_i)  # 综合评价值

In [12]:
result=pd.DataFrame(stander_data.T.index)

In [13]:
result['TOPSIS法得分']=score

In [14]:
result=result.sort_values("TOPSIS法得分",ascending=0)

In [15]:
result['TOPSIS法排名']=[i+1 for i in range(len(result))]

In [16]:
result 

,属性(指标),TOPSIS法得分,TOPSIS法排名
0,唐山市,1,1
2,沧州市,0.354119,2
1,秦皇岛市,0,3


In [17]:
result.to_excel('./TOPSIS法排名.xlsx')